# E-commerce Sales Analysis - 2023 Performance Review

## Business Objectives

This analysis examines e-commerce sales performance for 2023 compared to 2022, focusing on:

- **Revenue Analysis**: Total revenue growth and trends
- **Product Performance**: Top-performing categories and sales distribution
- **Geographic Analysis**: Revenue performance by state/region
- **Customer Experience**: Delivery performance and customer satisfaction metrics
- **Operational Insights**: Order fulfillment rates and status distribution

The analysis uses configurable parameters to enable flexible time-based filtering and comparison studies.

## Data Loading & Configuration

Configure analysis parameters and load required modules.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from datetime import datetime
import warnings

# Custom modules
from data_loader import EcommerceDataLoader, load_and_process_data
from business_metrics import (
    calculate_revenue_metrics,
    calculate_monthly_growth_trend,
    calculate_average_order_value,
    calculate_order_count_metrics,
    calculate_product_category_performance,
    calculate_geographic_performance,
    calculate_delivery_performance_metrics,
    generate_business_summary
)

warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('default')
sns.set_palette("husl")

print("📊 E-commerce Sales Analysis - Setup Complete")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### Analysis Configuration

Set configurable parameters for flexible analysis. Modify these values to analyze different time periods.

In [ ]:
# Analysis Configuration Parameters
CURRENT_YEAR = 2023          # Primary analysis year
COMPARISON_YEAR = 2022       # Comparison baseline year
ANALYSIS_MONTH = None        # Set to 1-12 for monthly analysis, None for full year
DATA_PATH = 'ecommerce_data/'  # Path to data files

# Display configuration
print("🔧 Analysis Configuration:")
print(f"   Primary Year: {CURRENT_YEAR}")
print(f"   Comparison Year: {COMPARISON_YEAR}")
print(f"   Month Filter: {ANALYSIS_MONTH if ANALYSIS_MONTH else 'Full Year'}")
print(f"   Data Path: {DATA_PATH}")

## Data Preparation & Transformation

Load and process all e-commerce datasets, applying necessary transformations and creating analytical datasets.

In [ ]:
# Load and process data using our data_loader module
print("📁 Loading e-commerce datasets...")
loader, processed_data = load_and_process_data(DATA_PATH)

# Get data summary
data_summary = loader.get_data_summary()
print("\n📈 Dataset Summary:")
for dataset, info in data_summary.items():
    print(f"   {dataset}: {info['rows']:,} rows, {info['columns']} columns")
    if info['date_range']:
        print(f"      Date range: {info['date_range']['start'].date()} to {info['date_range']['end'].date()}")

In [ ]:
# Create comprehensive sales dataset for analysis
print("🔄 Creating analytical dataset...")

# Create sales dataset with delivered orders only
sales_data = loader.create_sales_dataset(
    year_filter=None,  # Load all years for comparison
    month_filter=ANALYSIS_MONTH,
    status_filter='delivered'
)

print(f"✅ Sales dataset created: {len(sales_data):,} delivered order items")
print(f"   Date range: {sales_data['order_purchase_timestamp'].min().date()} to {sales_data['order_purchase_timestamp'].max().date()}")
print(f"   Years available: {sorted(sales_data['year'].unique())}")

# Display sample of the dataset
print("\n📊 Sample of analytical dataset:")
sales_data.head()

### Data Quality Assessment

In [ ]:
# Check data quality and completeness
print("🔍 Data Quality Assessment:")

# Missing values analysis
missing_data = sales_data.isnull().sum()
missing_pct = (missing_data / len(sales_data)) * 100

quality_df = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_pct
}).round(2)

quality_issues = quality_df[quality_df['Missing_Percentage'] > 0].sort_values('Missing_Percentage', ascending=False)

if not quality_issues.empty:
    print("\n⚠️ Columns with missing data:")
    print(quality_issues)
else:
    print("✅ No missing data detected")

# Basic statistics
print(f"\n📈 Key Metrics:")
print(f"   Total Revenue: ${sales_data['price'].sum():,.2f}")
print(f"   Unique Orders: {sales_data['order_id'].nunique():,}")
print(f"   Unique Customers: {sales_data['customer_id'].nunique():,}")
print(f"   Product Categories: {sales_data['product_category_name'].nunique() if 'product_category_name' in sales_data.columns else 'N/A'}")

## Business Metrics Calculation

### Revenue Analysis

Comprehensive revenue analysis comparing current year performance to previous year baseline.

In [ ]:
# Calculate comprehensive revenue metrics
revenue_metrics = calculate_revenue_metrics(sales_data, CURRENT_YEAR, COMPARISON_YEAR)

print(f"💰 Revenue Analysis ({CURRENT_YEAR} vs {COMPARISON_YEAR})")
print("=" * 50)
print(f"Current Year ({CURRENT_YEAR}) Revenue: ${revenue_metrics['current_revenue']:,.2f}")
print(f"Previous Year ({COMPARISON_YEAR}) Revenue: ${revenue_metrics['previous_revenue']:,.2f}")
print(f"Revenue Growth: {revenue_metrics['revenue_growth']:.2f}%")

# Visualize revenue comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Revenue comparison bar chart
years = [COMPARISON_YEAR, CURRENT_YEAR]
revenues = [revenue_metrics['previous_revenue'], revenue_metrics['current_revenue']]
colors = ['#E74C3C' if revenue_metrics['revenue_growth'] < 0 else '#2ECC71', '#3498DB']

bars = ax1.bar(years, revenues, color=colors, alpha=0.8, edgecolor='white', linewidth=2)
ax1.set_title(f'Revenue Comparison: {COMPARISON_YEAR} vs {CURRENT_YEAR}', fontsize=14, fontweight='bold', pad=20)
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Revenue ($)', fontsize=12)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add value labels on bars
for bar, value in zip(bars, revenues):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(revenues) * 0.01,
             f'${value:,.0f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Growth indicator
growth_color = '#E74C3C' if revenue_metrics['revenue_growth'] < 0 else '#2ECC71'
growth_text = 'DECLINE' if revenue_metrics['revenue_growth'] < 0 else 'GROWTH'

ax2.text(0.5, 0.6, f"{revenue_metrics['revenue_growth']:.1f}%", ha='center', va='center', 
         transform=ax2.transAxes, fontsize=36, fontweight='bold', color=growth_color)
ax2.text(0.5, 0.3, growth_text, ha='center', va='center', 
         transform=ax2.transAxes, fontsize=16, fontweight='bold', color=growth_color)
ax2.set_title('Year-over-Year Growth', fontsize=14, fontweight='bold', pad=20)
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')

plt.tight_layout()
plt.show()

### Monthly Revenue Trends

Analysis of month-over-month performance patterns within the primary analysis year.

In [ ]:
# Calculate monthly growth trends (only if not filtering by specific month)
if ANALYSIS_MONTH is None:
    monthly_growth = calculate_monthly_growth_trend(sales_data, CURRENT_YEAR)
    
    # Calculate monthly revenue for visualization
    monthly_revenue = sales_data[sales_data['year'] == CURRENT_YEAR].groupby('month')['price'].sum()
    
    print(f"📊 Monthly Performance Analysis - {CURRENT_YEAR}")
    print("=" * 50)
    print(f"Average Monthly Growth: {monthly_growth.mean():.2f}%")
    print(f"Best Month: {monthly_growth.idxmax()} ({monthly_growth.max():.1f}% growth)")
    print(f"Worst Month: {monthly_growth.idxmin()} ({monthly_growth.min():.1f}% growth)")
    
    # Create monthly trend visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Monthly revenue trend
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    ax1.plot(monthly_revenue.index, monthly_revenue.values, marker='o', 
             linewidth=3, markersize=8, color='#3498DB')
    ax1.fill_between(monthly_revenue.index, monthly_revenue.values, alpha=0.3, color='#3498DB')
    
    ax1.set_title(f'Monthly Revenue Trend - {CURRENT_YEAR}', fontsize=16, fontweight='bold', pad=20)
    ax1.set_xlabel('Month', fontsize=12)
    ax1.set_ylabel('Revenue ($)', fontsize=12)
    ax1.set_xticks(range(1, 13))
    ax1.set_xticklabels(months)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    ax1.grid(True, alpha=0.3)
    
    # Monthly growth rate
    colors = ['#E74C3C' if x < 0 else '#2ECC71' for x in monthly_growth.values[1:]]
    ax2.bar(monthly_growth.index[1:], monthly_growth.values[1:], color=colors, alpha=0.8)
    ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    ax2.set_title(f'Month-over-Month Growth Rate - {CURRENT_YEAR}', fontsize=16, fontweight='bold', pad=20)
    ax2.set_xlabel('Month', fontsize=12)
    ax2.set_ylabel('Growth Rate (%)', fontsize=12)
    ax2.set_xticks(range(2, 13))
    ax2.set_xticklabels(months[1:])
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"📊 Monthly analysis skipped (filtering by month {ANALYSIS_MONTH})")

### Order Volume & Average Order Value Analysis

In [ ]:
# Calculate order metrics
order_metrics = calculate_order_count_metrics(sales_data, CURRENT_YEAR, COMPARISON_YEAR)
aov_metrics = calculate_average_order_value(sales_data, CURRENT_YEAR, COMPARISON_YEAR)

print(f"📦 Order Volume Analysis ({CURRENT_YEAR} vs {COMPARISON_YEAR})")
print("=" * 50)
print(f"Current Year Orders: {order_metrics['current_orders']:,}")
print(f"Previous Year Orders: {order_metrics['previous_orders']:,}")
print(f"Order Growth: {order_metrics['order_growth']:.2f}%")

print(f"\n💳 Average Order Value Analysis")
print("=" * 35)
print(f"Current Year AOV: ${aov_metrics['current_aov']:.2f}")
print(f"Previous Year AOV: ${aov_metrics['previous_aov']:.2f}")
print(f"AOV Growth: {aov_metrics['aov_growth']:.2f}%")

# Create combined visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Order count comparison
years = [COMPARISON_YEAR, CURRENT_YEAR]
orders = [order_metrics['previous_orders'], order_metrics['current_orders']]
colors = ['#E74C3C' if order_metrics['order_growth'] < 0 else '#2ECC71', '#3498DB']

bars1 = ax1.bar(years, orders, color=colors, alpha=0.8)
ax1.set_title('Total Orders Comparison', fontsize=14, fontweight='bold')
ax1.set_ylabel('Number of Orders')
for bar, value in zip(bars1, orders):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(orders) * 0.01,
             f'{value:,}', ha='center', va='bottom', fontweight='bold')

# AOV comparison
aovs = [aov_metrics['previous_aov'], aov_metrics['current_aov']]
colors_aov = ['#E74C3C' if aov_metrics['aov_growth'] < 0 else '#2ECC71', '#3498DB']

bars2 = ax2.bar(years, aovs, color=colors_aov, alpha=0.8)
ax2.set_title('Average Order Value Comparison', fontsize=14, fontweight='bold')
ax2.set_ylabel('Average Order Value ($)')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:.0f}'))
for bar, value in zip(bars2, aovs):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(aovs) * 0.01,
             f'${value:.0f}', ha='center', va='bottom', fontweight='bold')

# Order growth indicator
growth_color = '#E74C3C' if order_metrics['order_growth'] < 0 else '#2ECC71'
ax3.text(0.5, 0.6, f"{order_metrics['order_growth']:.1f}%", ha='center', va='center', 
         transform=ax3.transAxes, fontsize=24, fontweight='bold', color=growth_color)
ax3.text(0.5, 0.3, 'Order Growth', ha='center', va='center', 
         transform=ax3.transAxes, fontsize=12, fontweight='bold')
ax3.axis('off')

# AOV growth indicator
aov_growth_color = '#E74C3C' if aov_metrics['aov_growth'] < 0 else '#2ECC71'
ax4.text(0.5, 0.6, f"{aov_metrics['aov_growth']:.1f}%", ha='center', va='center', 
         transform=ax4.transAxes, fontsize=24, fontweight='bold', color=aov_growth_color)
ax4.text(0.5, 0.3, 'AOV Growth', ha='center', va='center', 
         transform=ax4.transAxes, fontsize=12, fontweight='bold')
ax4.axis('off')

plt.tight_layout()
plt.show()

### Product Category Performance Analysis

Analysis of revenue performance by product category, identifying top-performing categories and market share distribution.

In [ ]:
# Calculate product category performance
if 'product_category_name' in sales_data.columns:
    # Load products data for category analysis
    products_data = loader.raw_data['products']
    category_performance = calculate_product_category_performance(sales_data, products_data, CURRENT_YEAR)
    
    # Add percentage of total revenue
    category_performance['revenue_share'] = (
        category_performance['price'] / category_performance['price'].sum() * 100
    ).round(2)
    
    print(f"🛍️ Product Category Performance - {CURRENT_YEAR}")
    print("=" * 55)
    print(f"Total Categories: {len(category_performance)}")
    print(f"Top Category: {category_performance.iloc[0]['product_category_name']} (${category_performance.iloc[0]['price']:,.2f})")
    print(f"Top 3 Categories Account for: {category_performance.head(3)['revenue_share'].sum():.1f}% of revenue")
    
    # Display top categories
    print("\n🏆 Top 10 Product Categories by Revenue:")
    top_categories = category_performance.head(10)
    for i, row in top_categories.iterrows():
        print(f"   {i+1:2d}. {row['product_category_name']:<25} ${row['price']:>10,.2f} ({row['revenue_share']:>5.1f}%)")
    
    # Create visualizations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
    # Top 10 categories bar chart
    colors = plt.cm.Set3(np.linspace(0, 1, len(top_categories)))
    bars = ax1.barh(range(len(top_categories)), top_categories['price'], color=colors)
    
    ax1.set_yticks(range(len(top_categories)))
    ax1.set_yticklabels([name.replace('_', ' ').title() for name in top_categories['product_category_name']])
    ax1.set_xlabel('Revenue ($)', fontsize=12)
    ax1.set_title(f'Top 10 Product Categories by Revenue - {CURRENT_YEAR}', fontsize=14, fontweight='bold', pad=20)
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # Add value labels
    for i, (bar, value, share) in enumerate(zip(bars, top_categories['price'], top_categories['revenue_share'])):
        ax1.text(bar.get_width() + max(top_categories['price']) * 0.01, bar.get_y() + bar.get_height()/2,
                 f'${value:,.0f} ({share:.1f}%)', va='center', fontsize=9)
    
    # Revenue share pie chart (top 8 + others)
    pie_data = category_performance.head(8).copy()
    others_revenue = category_performance.iloc[8:]['price'].sum()
    others_share = category_performance.iloc[8:]['revenue_share'].sum()
    
    if others_revenue > 0:
        others_row = pd.DataFrame({
            'product_category_name': ['Others'],
            'price': [others_revenue],
            'revenue_share': [others_share]
        })
        pie_data = pd.concat([pie_data, others_row], ignore_index=True)
    
    wedges, texts, autotexts = ax2.pie(pie_data['revenue_share'], 
                                       labels=[name.replace('_', ' ').title() for name in pie_data['product_category_name']], 
                                       autopct='%1.1f%%', startangle=90, colors=colors)
    
    ax2.set_title(f'Revenue Share by Product Category - {CURRENT_YEAR}', fontsize=14, fontweight='bold', pad=20)
    
    # Enhance pie chart text
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️ Product category data not available in dataset")

### Geographic Performance Analysis

Revenue analysis by state/geographic region to identify top-performing markets and regional trends.

In [ ]:
# Calculate geographic performance
if 'customer_state' in sales_data.columns:
    orders_data = loader.processed_data['orders']
    customers_data = loader.raw_data['customers']
    
    geographic_performance = calculate_geographic_performance(
        sales_data, orders_data, customers_data, CURRENT_YEAR
    )
    
    # Add market share
    geographic_performance['market_share'] = (
        geographic_performance['price'] / geographic_performance['price'].sum() * 100
    ).round(2)
    
    print(f"🗺️ Geographic Performance Analysis - {CURRENT_YEAR}")
    print("=" * 55)
    print(f"States with Sales: {len(geographic_performance)}")
    print(f"Top State: {geographic_performance.iloc[0]['customer_state']} (${geographic_performance.iloc[0]['price']:,.2f})")
    print(f"Top 5 States Account for: {geographic_performance.head(5)['market_share'].sum():.1f}% of revenue")
    
    # Display top states
    print("\n🏆 Top 10 States by Revenue:")
    top_states = geographic_performance.head(10)
    for i, row in top_states.iterrows():
        print(f"   {i+1:2d}. {row['customer_state']:<3} ${row['price']:>12,.2f} ({row['market_share']:>5.1f}%)")
    
    # Create visualizations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
    # Top 15 states bar chart
    top_15_states = geographic_performance.head(15)
    colors = plt.cm.viridis(np.linspace(0, 1, len(top_15_states)))
    
    bars = ax1.bar(range(len(top_15_states)), top_15_states['price'], color=colors)
    ax1.set_xticks(range(len(top_15_states)))
    ax1.set_xticklabels(top_15_states['customer_state'], rotation=45, ha='right')
    ax1.set_ylabel('Revenue ($)', fontsize=12)
    ax1.set_title(f'Top 15 States by Revenue - {CURRENT_YEAR}', fontsize=14, fontweight='bold', pad=20)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # Add value labels on top bars only
    for i, (bar, value) in enumerate(zip(bars[:5], top_15_states['price'][:5])):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(top_15_states['price']) * 0.01,
                 f'${value:,.0f}', ha='center', va='bottom', fontsize=9, rotation=90)
    
    # Market concentration analysis
    cumulative_share = geographic_performance['market_share'].cumsum()
    ax2.plot(range(1, len(cumulative_share) + 1), cumulative_share, marker='o', linewidth=2, markersize=4)
    ax2.axhline(y=80, color='red', linestyle='--', alpha=0.7, label='80% of Revenue')
    ax2.axhline(y=50, color='orange', linestyle='--', alpha=0.7, label='50% of Revenue')
    
    ax2.set_xlabel('Number of States (Ranked by Revenue)', fontsize=12)
    ax2.set_ylabel('Cumulative Revenue Share (%)', fontsize=12)
    ax2.set_title(f'Market Concentration Analysis - {CURRENT_YEAR}', fontsize=14, fontweight='bold', pad=20)
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # Add annotations for key thresholds
    states_for_50 = (cumulative_share >= 50).idxmax() + 1
    states_for_80 = (cumulative_share >= 80).idxmax() + 1
    
    ax2.annotate(f'Top {states_for_50} states = 50%', 
                xy=(states_for_50, 50), xytext=(states_for_50 + 5, 30),
                arrowprops=dict(arrowstyle='->', color='orange', alpha=0.7),
                fontsize=10, ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Create US map visualization using Plotly
    print("\n🗺️ Creating US Revenue Heatmap...")
    
    fig = px.choropleth(
        geographic_performance,
        locations='customer_state',
        color='price',
        locationmode='USA-states',
        scope='usa',
        title=f'Revenue by State - {CURRENT_YEAR}',
        color_continuous_scale='Viridis',
        labels={'price': 'Revenue ($)', 'customer_state': 'State'}
    )
    
    fig.update_layout(
        title_font_size=16,
        title_x=0.5,
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='albers usa'
        ),
        width=1000,
        height=600
    )
    
    fig.show()
    
else:
    print("⚠️ Geographic data not available in dataset")

## Customer Experience Analysis

### Delivery Performance & Customer Satisfaction

Analysis of delivery performance metrics and their correlation with customer satisfaction scores.

In [ ]:
# Calculate delivery performance metrics
if 'review_score' in sales_data.columns and 'order_delivered_customer_date' in sales_data.columns:
    reviews_data = loader.processed_data['reviews'] if 'reviews' in loader.processed_data else loader.raw_data['reviews']
    
    delivery_metrics = calculate_delivery_performance_metrics(sales_data, reviews_data, CURRENT_YEAR)
    
    print(f"🚚 Delivery Performance Analysis - {CURRENT_YEAR}")
    print("=" * 50)
    print(f"Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
    print(f"Average Review Score: {delivery_metrics['avg_review_score']:.2f}/5.0")
    
    print(f"\n📊 Customer Satisfaction by Delivery Speed:")
    print(f"   Fast Delivery (1-3 days): {delivery_metrics['fast_delivery_score']:.2f}/5.0")
    print(f"   Standard Delivery (4-7 days): {delivery_metrics['standard_delivery_score']:.2f}/5.0")
    print(f"   Slow Delivery (8+ days): {delivery_metrics['slow_delivery_score']:.2f}/5.0")
    
    # Create delivery analysis visualizations
    # Prepare data for detailed analysis
    current_year_data = sales_data[sales_data['year'] == CURRENT_YEAR].copy()
    current_year_data['delivery_days'] = (
        pd.to_datetime(current_year_data['order_delivered_customer_date']) - 
        pd.to_datetime(current_year_data['order_purchase_timestamp'])
    ).dt.days
    
    # Categorize delivery speed
    def categorize_delivery(days):
        if pd.isna(days):
            return 'Unknown'
        elif days <= 3:
            return '1-3 days'
        elif days <= 7:
            return '4-7 days'
        else:
            return '8+ days'
    
    current_year_data['delivery_category'] = current_year_data['delivery_days'].apply(categorize_delivery)
    
    # Remove duplicates for order-level analysis
    order_level_data = current_year_data.drop_duplicates('order_id')
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Delivery time distribution
    delivery_counts = order_level_data['delivery_category'].value_counts()
    colors = ['#2ECC71', '#F39C12', '#E74C3C', '#95A5A6']
    
    wedges, texts, autotexts = ax1.pie(delivery_counts.values, labels=delivery_counts.index, 
                                       autopct='%1.1f%%', colors=colors, startangle=90)
    ax1.set_title(f'Delivery Time Distribution - {CURRENT_YEAR}', fontsize=14, fontweight='bold')
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    # Average review score by delivery category
    review_by_delivery = order_level_data.groupby('delivery_category')['review_score'].mean().sort_values(ascending=True)
    
    bars = ax2.barh(range(len(review_by_delivery)), review_by_delivery.values, 
                    color=['#E74C3C' if x < 4.0 else '#F39C12' if x < 4.5 else '#2ECC71' for x in review_by_delivery.values])
    ax2.set_yticks(range(len(review_by_delivery)))
    ax2.set_yticklabels(review_by_delivery.index)
    ax2.set_xlabel('Average Review Score')
    ax2.set_title(f'Customer Satisfaction by Delivery Speed - {CURRENT_YEAR}', fontsize=14, fontweight='bold')
    ax2.set_xlim(0, 5)
    
    # Add value labels
    for bar, value in zip(bars, review_by_delivery.values):
        ax2.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2,
                 f'{value:.2f}', va='center', fontweight='bold')
    
    # Review score distribution
    review_dist = order_level_data['review_score'].value_counts().sort_index()
    bars3 = ax3.bar(review_dist.index, review_dist.values, color='#3498DB', alpha=0.8)
    ax3.set_xlabel('Review Score')
    ax3.set_ylabel('Number of Orders')
    ax3.set_title(f'Review Score Distribution - {CURRENT_YEAR}', fontsize=14, fontweight='bold')
    ax3.set_xticks(range(1, 6))
    
    # Add value labels
    for bar in bars3:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + max(review_dist.values) * 0.01,
                 f'{int(height)}', ha='center', va='bottom')
    
    # Delivery time vs Review score scatter
    valid_data = order_level_data.dropna(subset=['delivery_days', 'review_score'])
    scatter = ax4.scatter(valid_data['delivery_days'], valid_data['review_score'], 
                         alpha=0.5, c=valid_data['review_score'], cmap='RdYlGn', s=20)
    
    ax4.set_xlabel('Delivery Days')
    ax4.set_ylabel('Review Score')
    ax4.set_title(f'Delivery Time vs Customer Satisfaction - {CURRENT_YEAR}', fontsize=14, fontweight='bold')
    ax4.set_ylim(0.5, 5.5)
    
    # Add trend line
    if len(valid_data) > 1:
        z = np.polyfit(valid_data['delivery_days'], valid_data['review_score'], 1)
        p = np.poly1d(z)
        ax4.plot(valid_data['delivery_days'], p(valid_data['delivery_days']), "r--", alpha=0.8, linewidth=2)
    
    plt.colorbar(scatter, ax=ax4, label='Review Score')
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️ Delivery or review data not available for customer experience analysis")

### Operational Performance

Analysis of order fulfillment rates and operational efficiency metrics.

In [ ]:
# Analyze order status distribution and operational metrics
orders_data = loader.processed_data['orders']

# Calculate order status distribution for current year
current_year_orders = orders_data[orders_data['year'] == CURRENT_YEAR]
status_distribution = current_year_orders['order_status'].value_counts(normalize=True) * 100

print(f"⚙️ Operational Performance - {CURRENT_YEAR}")
print("=" * 45)
print(f"Total Orders Processed: {len(current_year_orders):,}")
print(f"Successful Delivery Rate: {status_distribution.get('delivered', 0):.1f}%")
print(f"Cancellation Rate: {status_distribution.get('canceled', 0):.1f}%")
print(f"Return Rate: {status_distribution.get('returned', 0):.1f}%")

print(f"\n📊 Order Status Distribution:")
for status, percentage in status_distribution.sort_values(ascending=False).items():
    print(f"   {status.title():<12}: {percentage:>6.1f}%")

# Create operational performance visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

# Order status pie chart
status_colors = {
    'delivered': '#2ECC71',
    'shipped': '#3498DB',
    'canceled': '#E74C3C',
    'processing': '#F39C12',
    'pending': '#95A5A6',
    'returned': '#8E44AD'
}

colors = [status_colors.get(status, '#BDC3C7') for status in status_distribution.index]

wedges, texts, autotexts = ax1.pie(status_distribution.values, 
                                   labels=[s.title() for s in status_distribution.index], 
                                   autopct='%1.1f%%', colors=colors, startangle=90)

ax1.set_title(f'Order Status Distribution - {CURRENT_YEAR}', fontsize=14, fontweight='bold', pad=20)

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# Monthly order processing trends
monthly_status = current_year_orders.groupby(['month', 'order_status']).size().unstack(fill_value=0)
monthly_total = monthly_status.sum(axis=1)
monthly_delivery_rate = (monthly_status.get('delivered', 0) / monthly_total * 100)

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax2.plot(monthly_delivery_rate.index, monthly_delivery_rate.values, 
         marker='o', linewidth=3, markersize=8, color='#2ECC71')
ax2.fill_between(monthly_delivery_rate.index, monthly_delivery_rate.values, 
                 alpha=0.3, color='#2ECC71')

ax2.set_title(f'Monthly Delivery Success Rate - {CURRENT_YEAR}', fontsize=14, fontweight='bold', pad=20)
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Delivery Rate (%)', fontsize=12)
ax2.set_xticks(range(1, 13))
ax2.set_xticklabels(months)
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3)

# Add target line
ax2.axhline(y=95, color='red', linestyle='--', alpha=0.7, label='Target (95%)')
ax2.legend()

plt.tight_layout()
plt.show()

## Summary of Key Observations

### Executive Summary

This section provides a comprehensive summary of key findings and actionable insights from the analysis.

In [ ]:
# Generate comprehensive business summary
try:
    business_summary = generate_business_summary(
        sales_data,
        loader.raw_data['products'],
        loader.processed_data['orders'],
        loader.raw_data['customers'],
        loader.raw_data['reviews'] if 'reviews' in loader.raw_data else pd.DataFrame(),
        current_year=CURRENT_YEAR,
        comparison_year=COMPARISON_YEAR,
        filter_month=ANALYSIS_MONTH
    )
    
    print("📋 EXECUTIVE SUMMARY")
    print("=" * 60)
    print(f"Analysis Period: {CURRENT_YEAR} vs {COMPARISON_YEAR}")
    if ANALYSIS_MONTH:
        print(f"Focus: Month {ANALYSIS_MONTH}")
    
    # Revenue Performance
    revenue = business_summary['revenue']
    print(f"\n💰 REVENUE PERFORMANCE:")
    print(f"   Current Revenue: ${revenue['current_revenue']:,.2f}")
    print(f"   Previous Revenue: ${revenue['previous_revenue']:,.2f}")
    growth_status = "📈 GROWTH" if revenue['revenue_growth'] > 0 else "📉 DECLINE"
    print(f"   Performance: {revenue['revenue_growth']:+.2f}% {growth_status}")
    
    # Order Metrics
    orders = business_summary['orders']
    aov = business_summary['aov']
    print(f"\n📦 ORDER METRICS:")
    print(f"   Total Orders: {orders['current_orders']:,} ({orders['order_growth']:+.1f}%)")
    print(f"   Average Order Value: ${aov['current_aov']:.2f} ({aov['aov_growth']:+.1f}%)")
    
    # Top Performing Categories
    if not business_summary['category_performance'].empty:
        top_categories = business_summary['category_performance'].head(3)
        print(f"\n🏆 TOP PRODUCT CATEGORIES:")
        for i, row in top_categories.iterrows():
            print(f"   {i+1}. {row['product_category_name'].replace('_', ' ').title()}: ${row['price']:,.2f}")
    
    # Geographic Leaders
    if not business_summary['geographic_performance'].empty:
        top_states = business_summary['geographic_performance'].head(3)
        print(f"\n🗺️ TOP GEOGRAPHIC MARKETS:")
        for i, row in top_states.iterrows():
            print(f"   {i+1}. {row['customer_state']}: ${row['price']:,.2f}")
    
    # Customer Experience
    if 'delivery_performance' in business_summary and business_summary['delivery_performance']:
        delivery = business_summary['delivery_performance']
        if 'avg_delivery_days' in delivery and not pd.isna(delivery['avg_delivery_days']):
            print(f"\n🚚 CUSTOMER EXPERIENCE:")
            print(f"   Average Delivery Time: {delivery['avg_delivery_days']:.1f} days")
            print(f"   Customer Satisfaction: {delivery['avg_review_score']:.2f}/5.0")
            
            # Delivery speed impact
            if delivery['fast_delivery_score'] > 0 and delivery['slow_delivery_score'] > 0:
                score_diff = delivery['fast_delivery_score'] - delivery['slow_delivery_score']
                print(f"   Fast vs Slow Delivery Impact: {score_diff:+.2f} points")
    
    # Operational Efficiency
    if 'order_status_distribution' in business_summary:
        status_dist = business_summary['order_status_distribution']
        if 'delivered' in status_dist.index:
            print(f"\n⚙️ OPERATIONAL EFFICIENCY:")
            print(f"   Delivery Success Rate: {status_dist['delivered']:.1f}%")
            if 'canceled' in status_dist.index:
                print(f"   Order Cancellation Rate: {status_dist['canceled']:.1f}%")
    
    # Key Insights and Recommendations
    print(f"\n💡 KEY INSIGHTS & RECOMMENDATIONS:")
    
    if revenue['revenue_growth'] < 0:
        print("   • Revenue decline requires immediate attention")
        print("   • Focus on customer retention and acquisition strategies")
    else:
        print("   • Strong revenue growth indicates healthy business performance")
    
    if aov['aov_growth'] < orders['order_growth']:
        print("   • Consider strategies to increase average order value")
        print("   • Implement cross-selling and upselling initiatives")
    
    if 'delivery_performance' in business_summary and business_summary['delivery_performance']:
        delivery = business_summary['delivery_performance']
        if 'avg_delivery_days' in delivery and delivery['avg_delivery_days'] > 7:
            print("   • Delivery times exceed 7 days - consider logistics optimization")
        if 'fast_delivery_score' in delivery and delivery['fast_delivery_score'] > delivery.get('slow_delivery_score', 0) + 0.2:
            print("   • Fast delivery significantly improves customer satisfaction")
    
    print("\n" + "=" * 60)
    
except Exception as e:
    print(f"⚠️ Error generating business summary: {str(e)}")
    print("\nManual Summary Based on Available Data:")
    print("=" * 40)
    
    # Fallback manual summary
    current_data = sales_data[sales_data['year'] == CURRENT_YEAR]
    previous_data = sales_data[sales_data['year'] == COMPARISON_YEAR]
    
    current_revenue = current_data['price'].sum()
    previous_revenue = previous_data['price'].sum()
    growth = ((current_revenue - previous_revenue) / previous_revenue * 100) if previous_revenue > 0 else 0
    
    print(f"Revenue {CURRENT_YEAR}: ${current_revenue:,.2f}")
    print(f"Revenue {COMPARISON_YEAR}: ${previous_revenue:,.2f}")
    print(f"Growth: {growth:+.2f}%")
    print(f"Total Orders {CURRENT_YEAR}: {current_data['order_id'].nunique():,}")

## Analysis Methodology & Configuration

### Data Sources
- **Orders Dataset**: Order-level information including status, timestamps, and customer IDs
- **Order Items Dataset**: Item-level details including product IDs, prices, and freight costs
- **Products Dataset**: Product catalog with categories and specifications
- **Customers Dataset**: Customer geographic information
- **Reviews Dataset**: Customer satisfaction scores and review data

### Key Metrics Definitions
- **Revenue Growth**: Year-over-year percentage change in total revenue
- **Average Order Value (AOV)**: Total revenue divided by number of unique orders
- **Delivery Performance**: Days between order placement and customer delivery
- **Customer Satisfaction**: Average review score (1-5 scale)
- **Order Fulfillment Rate**: Percentage of orders successfully delivered

### Configurable Parameters
This analysis is designed to be flexible and configurable:
- **Time Period**: Modify `CURRENT_YEAR`, `COMPARISON_YEAR`, and `ANALYSIS_MONTH` variables
- **Data Filtering**: Adjust filters in data loading functions
- **Visualization Preferences**: Customize charts and color schemes

### Future Enhancements
- Customer segmentation analysis
- Seasonal trend analysis
- Cohort analysis for customer retention
- Predictive analytics for demand forecasting